In [4]:
import sys
import asyncio
#sys.path.append('/Users/andreas/fun/LED/pyTibber')
sys.path.append('/home/andram/fun/LED-env/lib/python3.11/site-packages/tibber')
from tibber import Tibber


In [11]:
async def main():
    TOKEN = "U4L8yS_OHsfgKndAhNQZ8K-JYElbNUagYvToCF3ZPVE"
    USER_AGENT = "LED_client"
    tibber_connection = Tibber(access_token=TOKEN,user_agent=USER_AGENT)
    await tibber_connection.update_info()

    # GraphQL query to fetch home information
    query = '''
    {
      viewer {
        homes {
          id
          address {
            address1
            city
          }
          owner {
            name
          }
          currentSubscription {
            priceInfo {
              current {
                total
                level
              }
            }
          }
        }
      }
    }
    '''

    data = await tibber_connection.execute(query)
    if data:
        homes = data['viewer']['homes']
        for home in homes:
            print(f"Home ID: {home['id']}")
            print(f"Address: {home['address']['address1']}, {home['address']['city']}")
            print(f"Owner's Name: {home['owner']['name']}")

            # Check if currentSubscription and priceInfo are not None
            if home.get('currentSubscription') and home['currentSubscription'].get('priceInfo'):
                current_price_info = home['currentSubscription']['priceInfo'].get('current')
                if current_price_info:
                    print(f"Current Power Price: {current_price_info['total']}")
                    print(f"Price Level: {current_price_info['level']}")
            print()

    await tibber_connection.close_connection()
await main()

Home ID: f65b179b-f74f-492c-9ea8-db1e1461856a
Address: Helmers vei 10, LAKSEVÅG
Owner's Name: Erlend Thue

Home ID: 975996b6-e7ca-4fbf-9f72-61e2df95bc0c
Address: Løbergsalleen 16, BERGEN
Owner's Name: Erlend Thue
Current Power Price: 0.8826
Price Level: CHEAP



In [23]:
import aiohttp
import asyncio
import json

async def fetch_websocket_url(token):
    graphql_endpoint = "https://api.tibber.com/v1-beta/gql"
    query = {
        "query": "{ viewer { websocketSubscriptionUrl } }"
    }
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }

    async with aiohttp.ClientSession() as session:
        async with session.post(graphql_endpoint, headers=headers, json=query) as response:
            if response.status == 200:
                data = await response.json()
                return data["data"]["viewer"]["websocketSubscriptionUrl"]
            else:
                raise Exception("Failed to fetch WebSocket URL")

# Use this function to get the WebSocket URL
TOKEN = "U4L8yS_OHsfgKndAhNQZ8K-JYElbNUagYvToCF3ZPVE"
websocket_url = await fetch_websocket_url(TOKEN)
print("WebSocket URL:", websocket_url)


WebSocket URL: wss://websocket-api.tibber.com/v1-beta/gql/subscriptions


In [26]:

TOKEN = "U4L8yS_OHsfgKndAhNQZ8K-JYElbNUagYvToCF3ZPVE" # Replace with your actual Tibber access token

def data_callback(data):
    print("Real-time data received:", data)
    # Process the real-time data here
    # Example: Update global variables or trigger actions

async def run():
    async with Tibber(TOKEN, user_agent="Ancreas Ramsli") as tibber_connection:
        await tibber_connection.update_info()
        home = tibber_connection.get_homes()[0]
        
        # Subscribe to real-time updates
        await home.rt_subscribe(data_callback)

        # Keep the connection open
        while True:
            await asyncio.sleep(10)

await run()


Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7f70fff750>


TypeError: 'Tibber' object does not support the asynchronous context manager protocol

In [16]:
import asyncio
import websockets
import json

# Global variable for Price Level
global_price_level = None

async def listen_for_price_changes(uri, token):
    global global_price_level

    async with websockets.connect(uri) as websocket:
        # Authenticate
        await websocket.send(json.dumps({
            "type": "connection_init",
            "payload": {"Authorization": f"Bearer {token}"}
        }))
        await websocket.recv()  # Wait for acknowledgement

        # Subscribe to price level updates
        query = '''
        subscription {
          liveMeasurement(homeId: "YOUR_HOME_ID") {
            timestamp
            level
          }
        }
        '''
        await websocket.send(json.dumps({"id": "1", "type": "start", "payload": {"query": query}}))

        # Listen for messages
        while True:
            response = await websocket.recv()
            data = json.loads(response)
            if data.get("type") == "data":
                new_price_level = data["payload"]["data"]["liveMeasurement"]["level"]
                if new_price_level != global_price_level:
                    global_price_level = new_price_level
                    print(f"Price Level Updated: {global_price_level}")
                    # Here you can add additional logic to communicate with external systems

async def main():
    TOKEN = "U4L8yS_OHsfgKndAhNQZ8K-JYElbNUagYvToCF3ZPVE"
    WEBSOCKET_URL = "wss://api.tibber.com/v1-beta/gql/subscriptions"  # Replace with the actual WebSocket URL from Tibber

    # Start listening for price changes
    await listen_for_price_changes(WEBSOCKET_URL, TOKEN)

main()


<coroutine object main at 0x7f70280c80>